In [19]:
!pip install llama-index-vector-stores-lancedb
!pip install llama-index-multi-model-llms-openai
!pip install llama-index-embeddings-clip
!pip install git+https://github.com/openai/CLIP.git
!pip install llama-index-readers-file


ERROR: Could not find a version that satisfies the requirement llama-index-multi-model-llms-openai (from versions: none)
ERROR: No matching distribution found for llama-index-multi-model-llms-openai
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-gvclpwtm
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-gvclpwtm
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [20]:
!pip install llama-index
!pip install -U openai-whisper

In [21]:
!pip install lancedb
!pip install moviepy
!pip install pytube
!pip install pydub
!pip install SpeechRecognition
!pip install ffmpeg-python
!pip install soundfile
!pip install torch torchvision
!pip install matplotlib scikit-image
!pip install ftfy regex tqdm

In [22]:
from moviepy.editor import VideoFileClip
from pathlib import Path
import os
import speech_recognition as sr
from pytube import YouTube
from pprint import pprint
from PIL import Image
import matplotlib.pyplot as plt

In [23]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [24]:
import os
print(os.getcwd())

/content


# Download Video from youtube

In [25]:
video_url="https://www.youtube.com/watch?v=NRmAXDWJVnU&t=3s"
#video_url="https://youtu.be/3dhcmeOTZ_Q"

In [26]:
output_video_path = '/content/video_data/'

# From the Video we will be collecting Images, Audio, Text

In [27]:
output_folder = '/content/mixed_data/'
output_audio_path = '/content/mixed_data/output_audio.wav'

In [28]:
!mkdir mixed_data

mkdir: cannot create directory ‘mixed_data’: File exists


In [29]:
filepath = output_video_path + "input_vid.mp4"
print(filepath)

/content/video_data/input_vid.mp4


# Let's download the Youtube Video and extract the MetaData

In [30]:
!pip install yt-dlp

In [31]:
import yt_dlp

def download_video_yt_dlp(url, output_path):
    ydl_opts = {
        'outtmpl': f'{output_path}/input_vid.%(ext)s',  # Saves as input_vid.mp4 (or .webm, depending)
        'format': 'bestvideo+bestaudio/best',
        'merge_output_format': 'mp4',  # force mp4 output
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        metadata = {
            "Author": info.get("uploader"),
            "Title": info.get("title"),
            "Views": info.get("view_count")
        }
        return metadata


# Collect Images from Video

In [32]:
from moviepy.editor import VideoFileClip
def video_to_images(video_path,output_folder):
  clip=VideoFileClip(video_path)
  clip.write_images_sequence(
      os.path.join(output_folder,"frame%04d.png"),fps=0.2
  )

# Process Audio from Vedio itself

In [33]:
def video_to_audio(video_path,output_audio_path):
  clip=VideoFileClip(video_path)
  audio=clip.audio
  audio.write_audiofile(output_audio_path)

# Collect Texts from Audio

In [34]:

def audio_to_text(audio_path):
  recognizer=sr.Recognizer()
  audio=sr.AudioFile(audio_path)

  with audio as source:
    audio_data=recognizer.record(source)

    try:

      #recognize the speech
      text = recognizer.recognize_whisper(audio_data)

    except sr.UnknownValueError:
      print("Speech recognition could not understand the audio.")
  return text

In [35]:
video_url

'https://www.youtube.com/watch?v=NRmAXDWJVnU&t=3s'

In [36]:
metadata_video = download_video_yt_dlp(video_url, output_video_path)
print(metadata_video)

[youtube] Extracting URL: https://www.youtube.com/watch?v=NRmAXDWJVnU&t=3s
[youtube] NRmAXDWJVnU: Downloading webpage
[youtube] NRmAXDWJVnU: Downloading tv client config
[youtube] NRmAXDWJVnU: Downloading tv player API JSON
[youtube] NRmAXDWJVnU: Downloading ios player API JSON
[youtube] NRmAXDWJVnU: Downloading m3u8 information
[info] Testing format 616
[info] NRmAXDWJVnU: Downloading 1 format(s): 616+251
[download] /content/video_data//input_vid.mp4 has already been downloaded
{'Author': 'Simplilearn', 'Title': 'Generative AI Explained In 5 Minutes | What Is GenAI? | Introduction To Generative AI | Simplilearn', 'Views': 921198}


In [37]:
video_to_images(filepath,output_folder)

Moviepy - Writing frames /content/mixed_data/frame%04d.png.


Moviepy - Done writing frames /content/mixed_data/frame%04d.png.


In [38]:
video_to_audio(filepath,output_audio_path)

MoviePy - Writing audio in /content/mixed_data/output_audio.wav


MoviePy - Done.


In [39]:
text_data = audio_to_text(output_audio_path)

#Let's write this text data inside one text file

In [40]:
with open(output_folder+'output_text.txt','w') as file:
  file.write(text_data)
print("Text data saved to file")
file.close()

Text data saved to file


# Now Multimodel Actual Activity would start

### In order to create embeddings, since we already have audio extracted from vedio, and even text extracted from audio, which would finally be the used for embeddings.(Now do not need audio file further).

# we will have to create our RAG using processing the video,Images and Text only

In [41]:
! pip install --upgrade llama-index


In [42]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext

In [43]:
from llama_index.vector_stores.lancedb import LanceDBVectorStore

In [44]:
text_store = LanceDBVectorStore(uri="lancedb",table_name="text_collection")
image_store = LanceDBVectorStore(uri="lancedb",table_name="image_collection")

In [45]:
storage_context = StorageContext.from_defaults(vector_store=text_store, image_store=image_store)

In [46]:
output_folder

'/content/mixed_data/'

In [47]:
documents = SimpleDirectoryReader(output_folder).load_data()

In [48]:
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

In [49]:
retriever_engine = index.as_retriever(similarity_top_k=1, image_similarity_top_k=3)

In [50]:
qa_template_str= (
    "Based on the Provided Information , including relevant images and retreived context from the video,\
    accurately and precisely answer  the query without any additional prior knowledge\n"

    "--------------------------------\n"
    "Context: {context_str}\n"
    "Metadata for video : {metadata_str} \n"

    "--------------------------------\n"
    "Query: {query_str}\n"
    "Answer: "

)

In [51]:
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.schema import ImageNode

In [52]:
def retrieve(retriever_engine, query_str):
    retrieval_results = retriever_engine.retrieve(query_str)

    retrieved_image = []
    retrieved_text = []
    for res_node in retrieval_results:
        if isinstance(res_node.node, ImageNode):
            retrieved_image.append(res_node.node.metadata["file_path"])
        else:
            display_source_node(res_node, source_length=200)
            retrieved_text.append(res_node.text)

    return retrieved_image, retrieved_text

In [53]:
query="what Generative AI primarily do?"

In [54]:
img,text=retrieve(retriever_engine,query)

**Node ID:** 80c2912a-753b-43b5-9592-64a386a6b57c<br>**Similarity:** 0.7854102849960327<br>**Text:** Meet Emma, a graphic designer working on a new project. One day, her colleague mentions a tool that helps create designs, images, and text using AI. Intrigued, Emma wonders how AI can create someth...<br>

In [55]:
img

[]

In [56]:
import matplotlib.pyplot as plt
def plot_images(images_path):
  images_shown = 0
  plt.figure(figsize=(16, 9))
  for img_path in images_path:
        if os.path.isfile(img_path):
            image = Image.open(img_path)

            plt.subplot(2, 3, images_shown + 1)
            plt.imshow(image)
            plt.xticks([])
            plt.yticks([])

            images_shown += 1
            if images_shown >= 5:
                break

In [57]:
plot_images(img)

<Figure size 1600x900 with 0 Axes>

In [58]:
!pip install llama-index-multi-modal-llms-openai

In [59]:
from llama_index.multi_modal_llms.openai import OpenAIMultiModal

In [60]:
openai_mm_llm = OpenAIMultiModal(model="gpt-4-vision-preview", api_key=OPENAI_API_KEY, max_new_tokens=1500)

  openai_mm_llm = OpenAIMultiModal(model="gpt-4-vision-preview", api_key=OPENAI_API_KEY, max_new_tokens=1500)



In [68]:
context_str=''.join(text)

In [70]:
image_documents = SimpleDirectoryReader(input_dir=output_folder,input_files=img).load_data()

In [71]:
import json
metadata_str = json.dumps(metadata_video)

In [74]:
result=openai_mm_llm.complete(
    prompt=qa_template_str.format(
        query_str=query,metadata_str=metadata_str, context_str=context_str
    ),
    image_documents=image_documents,
)

ValueError: Unsupported content block type: ImageDocument